In [29]:
# imports

import os
import requests
import xml.etree.ElementTree as ET
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display


# If you get an error running this cell, then please head over to the troubleshooting notebook!

In [3]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


In [4]:
openai = OpenAI()


In [14]:
# url = "https://www.aemet.es/xml/municipios/localidad_03066.xml"
# response = requests.get(url)
# root = ET.fromstring(response.content)

class XMLCityWeather:
    def __init__(self, url):
        self.url = url
        self.root = ET.fromstring(requests.get(url).content)

    def xml(self):
        return ET.tostring(self.root, encoding='unicode')

    def get_city(self):
        return self.root.find('nombre').text + " (" + self.root.find('provincia').text + ")"


In [19]:
climbing_schools = [
  {
    "name": "Marin",
    "xml_url": "https://www.aemet.es/xml/municipios/localidad_03066.xml",
  },
  {
    "name": "Sax",
    "xml_url": "https://www.aemet.es/xml/municipios/localidad_03123.xml",
  },
  {
    "name": "Aspe",
    "xml_url": "https://www.aemet.es/xml/municipios/localidad_03019.xml",
  }
]

elda = XMLCityWeather(climbing_schools[0]["xml_url"])
sax = XMLCityWeather(climbing_schools[1]["xml_url"])
aspe = XMLCityWeather(climbing_schools[2]["xml_url"])
alicante_climbing_schools = [elda, sax, aspe]


In [24]:
system_prompt = "You are an assistant that analyzes the xml of a city with weather information \
and provides a short summary of the weather for the next 7 days from the perspective of a rock climber. \
Respond in markdown."

def user_prompt_for(climbing_schools):
    user_prompt = f"You are looking a xml with weather information for a climbing school"
    user_prompt += "\nThe contents of this xml is as follows; \
please provide a short summary of this xml in markdown. \
And provider the best climbing school based on the weather information for the next 7 days.\n\n"
    for school in climbing_schools:
        user_prompt += school.get_city() + "\n"
        user_prompt += school.xml() + "\n\n"
    return user_prompt



In [25]:
def messages_for(climbing_schools):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(climbing_schools)}
    ]

In [26]:
def best_climbing_school(climbing_schools):
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(climbing_schools)
    )
    return response.choices[0].message.content

In [30]:
def display_summary(climbing_schools):
    summary = best_climbing_school(climbing_schools)
    display(Markdown(summary))

display_summary(alicante_climbing_schools)

# Weather Summary for Climbers in Elda, Sax, and Aspe (April 2025)

## Elda
- **March 31**: Clear skies with temperatures ranging from 6°C to 25°C. Excellent day for climbing, as no precipitation is expected.
- **April 1**: Mostly sunny with temperatures of 9°C to 23°C. Very favorable conditions for climbing.
- **April 2**: Expect cloudy skies with a 55% chance of light rain. Temperatures from 9°C to 19°C; climbing may be impacted.
- **April 3**: Intervals of clouds with a 35% chance of rain. Temperatures between 10°C and 23°C, potentially manageable for climbing with good equipment.
- **April 4**: 70% chance of rain with cloudy skies. Not ideal for climbing, with high humidity levels.
- **April 5**: Only a 10% chance of precipitation, great intervals of clouds, and temperatures between 10°C and 22°C. Good climbing day.
- **April 6**: A 20% chance of rain, and conditions remain reasonable for climbing with temperatures up to 22°C.

### **Best Day for Climbing in Elda:** April 1

## Sax
- **March 31**: Clear skies throughout the day, temperatures between 5°C and 25°C. Perfect conditions for climbing.
- **April 1**: Similar to Elda, expect sunny skies with temps from 7°C to 23°C. Ideal for outdoor activities.
- **April 2**: 60% chance of rain with cloudy weather. Temperatures between 8°C and 19°C; best to check the weather closely.
- **April 3**: 35% chance of rain but mostly overcast. Temperatures ranging from 9°C to 22°C; climbing may be possible but less favorable.
- **April 4**: High probability of light rain (75%). It's best to avoid climbing.
- **April 5**: Low chance of rain (10%). Ideal conditions with temperatures between 8°C and 21°C.
- **April 6**: 25% chance of rain. A relatively good day to climb with temps reaching 21°C.

### **Best Day for Climbing in Sax:** April 1

## Aspe
- **March 31**: Clear and sunny with temperatures from 7°C to 26°C. Excellent climbing weather.
- **April 1**: Generally sunny with some high clouds. Temperatures between 11°C and 23°C; perfect for climbing.
- **April 2**: 45% chance of precipitation and a mostly cloudy day. Highs around 20°C can make conditions tricky.
- **April 3**: 25% chance of rain in the morning with slightly improving conditions in the afternoon. Temps between 9°C and 23°C.
- **April 4**: A 60% chance of rain means climbing is discouraged.
- **April 5**: Low precipitation chance (10%), breaking clouds, with temperatures reaching 25°C—not bad for climbing.
- **April 6**: A 20% chance of rain and temperatures reaching 24°C, making it another day with potential for climbing.

### **Best Day for Climbing in Aspe:** April 1

## **Conclusion: Best Climbing School**
Based on the weather forecast for the next week, **Sax** appears to have the best overall conditions for outdoor climbing, especially on April 1. The lack of precipitation and favorable temperatures make it an excellent choice for climbers looking for ideal weather conditions.